In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pickle
import mne

from mne.channels import make_1020_channel_selections
from mne.event import define_target_events
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


from sklearn.model_selection import KFold, train_test_split, cross_validate
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")


from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

class_names = ['1','2','3']

In [18]:
os.chdir('/home/castrogaray-j/U_Winnipeg_OneDrive/Pattern recognition/BICT/Events_raw')
'''test'''

'test'

In [19]:
print('more test')

more test


In [20]:

files = os.listdir()
pkl_files = sorted([file for file in files if file.startswith('raw') and file.endswith('.pkl')])
subjects = set([file[4:6] for file in pkl_files])
#pkl_files
#len(pkl_files)
#int(pkl_files[9][8:10])

In [21]:
 with open(pkl_files[0], 'rb') as file:
     events = pickle.load(file)
events[0]

<RawEEGLAB | E:\Datasets\BCIT_Mind_Wandering\sub-01\ses-01\eeg\sub-01_ses-01_task-DriveWithTaskAudio_run-1_eeg.set, 64 x 5121 (5.0 s), ~2.6 MB, data loaded>

In [22]:
wind_length = 5
b_freq, u_freq = 1.0, 50.0

freq_bands = {
        'Delta': (1, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
        'Gamma': (30, 50)
    }

def compute_PSD(raw, file):
    sfreq = raw.info['sfreq']

    epochs = mne.make_fixed_length_epochs(raw, duration=wind_length, preload=True, verbose='error')
    
    
    features = np.empty([ epochs.get_data().shape[0], 320 ] )
    for idx, epoch_data in enumerate( epochs.get_data() * 1e6 ):
            #print(epoch_data.shape  ) 
            #cogemos los segmentos y los dividimos en 4 segundos
            psds, freqs = mne.time_frequency.psd_array_welch( epoch_data, sfreq=sfreq, fmin=b_freq, fmax=u_freq,
                                                             n_fft = 4096, verbose='error' )#, n_per_seg= 2048 )
            
            
            band_power_list= [] 
    
            
            for band, (fmin, fmax) in freq_bands.items():
                #print( fmin, fmax ) 
                freq_idx = np.logical_and(freqs>=fmin, freqs<=fmax)
                band_power = psds[:, freq_idx].mean(axis=1)
                # esto tiene dimensiones 64, 1
                band_power_list.append(band_power.reshape(-1,1) )
            #print( band_power_list[0].shape  )
        
            
            band_power_list = np.concatenate(band_power_list ,axis = 1 )
            #print( 'matrix 64, 5', band_power_list.shape  )
    
            
            features[idx] = band_power_list.reshape(1,-1)
        
    df = pd.DataFrame(features)
    df['class'] = file[-9]

    #df.to_csv('Features\\{}_class_{}.csv'.format(file[50:56], file[-9]), index=False)

    return df

In [23]:
sfreq = events[0].info['sfreq']
wind_length = 4
b_freq, u_freq = 1.0, 50.0
epochs = mne.make_fixed_length_epochs(events[0], duration=wind_length, preload=True, verbose='error')
freqs = np.arange(b_freq, u_freq, 1)  # Frequency range
n_cycles = freqs / 2.  # Number of cycles per frequency
power = mne.time_frequency.tfr_multitaper(
    epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False
)



# Plot the spectrogram
#power.plot(baseline=(None, 0), mode='logratio', title='EEG Spectrogram')

#plt.show()

NOTE: tfr_multitaper() is a legacy function. New code should use .compute_tfr(method="multitaper").


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


In [24]:
%%capture --no-stderr --no-stdout
epochs[0]
power = mne.time_frequency.tfr_multitaper(
    epochs[0], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False
)
fig = power.plot(baseline=(None, 0), mode='logratio',colorbar=False)
# Quitar ejes
for ax in fig[0].axes:
    ax.set_xticks([])  # Quitar ticks en eje X
    ax.set_yticks([])  # Quitar ticks en eje Y
    ax.set_xlabel("")  # Quitar etiqueta del eje X
    ax.set_ylabel("")  # Quitar etiqueta del eje Y
    ax.set_frame_on(False)  # Quitar borde del gráfico
fig[0].show()
fig[0].savefig("/home/castrogaray-j/Spectrogram_images/spectrogram_no_axes.png", dpi=100)

NOTE: tfr_multitaper() is a legacy function. New code should use .compute_tfr(method="multitaper").


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Applying baseline correction (mode: logratio)


In [25]:
epochs

<Epochs | 1 events (all good), 0 – 3.999 s (baseline off), ~2.1 MB, data loaded,
 '1': 1>

In [26]:
'''%%capture --no-stderr --no-stdout
wind_length = 4
b_freq, u_freq = 1.0, 50.0
sfreq = events[0].info['sfreq']
freqs = np.arange(b_freq, u_freq, 1)  # Frequency range
n_cycles = freqs / 2.  # Number of cycles per frequency

ch_names = epochs[0].info['ch_names']

total_iter= len(events)
progress_bar = tqdm(total=total_iter, desc="Progress", unit="iter")

for idx, event in enumerate(events):
    copy_croped = event.copy().crop(tmin=1)
    epoch = mne.make_fixed_length_epochs(copy_croped, duration=wind_length, preload=True, verbose='error')
    for ch in ch_names:
        power = epoch.compute_tfr(method="multitaper", freqs=freqs, n_cycles=n_cycles, picks=ch)
        fig = power.plot(baseline=(None, 0), mode='logratio', show=False, verbose='error', colorbar=False);
        for ax in fig[0].axes:
            ax.set_xticks([])  # Quitar ticks en eje X
            ax.set_yticks([])  # Quitar ticks en eje Y
            ax.set_xlabel("")  # Quitar etiqueta del eje X
            ax.set_ylabel("")  # Quitar etiqueta del eje Y
            ax.set_frame_on(False)  # Quitar borde del gráfico
        fig[0].savefig("/home/castrogaray-j/Spectrogram_images/subject_{}_class_{}_event_{}_channel_{}.png".format(1, 2, idx, ch), dpi=100)
        fig.clear()

    progress_bar.update(1)

# Cerrar la barra de progreso al finalizar
progress_bar.close()

#
#plt.savefig('/home/castrogaray-j/Spectrogram_images/spectrogram.png', dpi=300)
#fig[0].savefig('/home/castrogaray-j/Spectrogram_images/spectrogram.png', dpi=100)'''

'%%capture --no-stderr --no-stdout\nwind_length = 4\nb_freq, u_freq = 1.0, 50.0\nsfreq = events[0].info[\'sfreq\']\nfreqs = np.arange(b_freq, u_freq, 1)  # Frequency range\nn_cycles = freqs / 2.  # Number of cycles per frequency\n\nch_names = epochs[0].info[\'ch_names\']\n\ntotal_iter= len(events)\nprogress_bar = tqdm(total=total_iter, desc="Progress", unit="iter")\n\nfor idx, event in enumerate(events):\n    copy_croped = event.copy().crop(tmin=1)\n    epoch = mne.make_fixed_length_epochs(copy_croped, duration=wind_length, preload=True, verbose=\'error\')\n    for ch in ch_names:\n        power = epoch.compute_tfr(method="multitaper", freqs=freqs, n_cycles=n_cycles, picks=ch)\n        fig = power.plot(baseline=(None, 0), mode=\'logratio\', show=False, verbose=\'error\', colorbar=False);\n        for ax in fig[0].axes:\n            ax.set_xticks([])  # Quitar ticks en eje X\n            ax.set_yticks([])  # Quitar ticks en eje Y\n            ax.set_xlabel("")  # Quitar etiqueta del eje

In [ ]:
%%capture --no-stderr --no-stdout
wind_length = 4
b_freq, u_freq = 1.0, 50.0
sfreq = events[0].info['sfreq']
freqs = np.arange(b_freq, u_freq, 1)  # Frequency range
n_cycles = freqs / 2.  # Number of cycles per frequency

ch_names = epochs[0].info['ch_names']

total_iter= len(pkl_files)
progress_bar = tqdm(total=total_iter, desc="Progress", unit="iter")

with open('event_labels/performance_labels', 'rb') as file:
     labels_array = pickle.load(file)
    
for i, pkl in enumerate(pkl_files):
    labels_df = labels_array[i]
    with open(pkl, 'rb') as file:
        events = pickle.load(file)
    filtered_events = [item for i, item in enumerate(events[1:]) if i in labels_df.index]
    
    for idx, event in enumerate(filtered_events):
        copy_croped = event.copy().crop(tmin=1)
        epoch = mne.make_fixed_length_epochs(copy_croped, duration=wind_length, preload=True, verbose='error')
        label = labels_df['label'].values[idx]
        for ch in ch_names:
            power = epoch.compute_tfr(method="multitaper", freqs=freqs, n_cycles=n_cycles, picks=ch)
            fig = power.plot(baseline=(None, 0), mode='logratio', show=False, verbose='error', colorbar=False);
            for ax in fig[0].axes:
                ax.set_xticks([])  # Quitar ticks en eje X
                ax.set_yticks([])  # Quitar ticks en eje Y
                ax.set_xlabel("")  # Quitar etiqueta del eje X
                ax.set_ylabel("")  # Quitar etiqueta del eje Y
                ax.set_frame_on(False)  # Quitar borde del gráfico
            fig[0].savefig("/home/castrogaray-j/Spectrogram_images/subject_{}_class_{}_label_{}_event_{}_channel_{}.png".format(int(pkl[8:10]), int(pkl[-5]), label, idx, ch), dpi=100)
            fig.clear()
    
        
        '''for j, event in enumerate(filtered_events):
            label = labels_df['label'].values[j]
            data = pd.DataFrame(event.get_data())
            data = data.transpose()
            data['class'] = int(pkl[-5])
            data['subject'] = int(pkl[8:10])
            data['label'] = label
            df_temp = pd.concat([df_temp, data], ignore_index=True)
            #print(' class: {}, subject: {}, label: {}'.format(int(pkl[-5]), int(pkl[8:10]), label))
    
        df = pd.concat([df, df_temp], ignore_index=True)'''
    progress_bar.update(1)

# Cerrar la barra de progreso al finalizar
progress_bar.close()


#
#plt.savefig('/home/castrogaray-j/Spectrogram_images/spectrogram.png', dpi=300)
#fig[0].savefig('/home/castrogaray-j/Spectrogram_images/spectrogram.png', dpi=100)


rogress:   0%|                                                                                                                                                                                                                                               | 0/56 [00:28<?, ?iter/s]

In [ ]:
total_iter= len(pkl_files)
progress_bar = tqdm(total=total_iter, desc="Progress", unit="iter")
df = pd.DataFrame()
with open('event_labels/performance_labels', 'rb') as file:
     labels_array = pickle.load(file)
    
for i, pkl in enumerate(pkl_files):
    labels_df = labels_array[i]

    with open(pkl, 'rb') as file:
        events = pickle.load(file)
    filtered_events = [item for i, item in enumerate(events[1:]) if i in labels_df.index]
    
    df_temp = pd.DataFrame()
    for j, event in enumerate(filtered_events):
        label = labels_df['label'].values[j]
        data = pd.DataFrame(event.get_data())
        data = data.transpose()
        data['class'] = int(pkl[-5])
        data['subject'] = int(pkl[8:10])
        data['label'] = label
        df_temp = pd.concat([df_temp, data], ignore_index=True)
        #print(' class: {}, subject: {}, label: {}'.format(int(pkl[-5]), int(pkl[8:10]), label))

    df = pd.concat([df, df_temp], ignore_index=True)
    progress_bar.update(1)

# Cerrar la barra de progreso al finalizar
progress_bar.close()

df



In [57]:
50*64*170

544000

In [22]:
from mne.time_frequency import tfr_morlet
power = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False)
power

NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Data type,Average Power Estimates
Data source,Epochs
Number of epochs,64
Dims,"channel, freq, time"
Estimation method,morlet
Number of channels,64
Number of timepoints,5120
Number of frequency bins,49
Frequency range,1.00 – 49.00 Hz


In [ ]:
# Plot single-channel spectrogram
power.plot( baseline=(None, 0), mode='logratio', title='Morlet Wavelet Spectrogram')

In [ ]:
total_iter= len(pkl_files)
progress_bar = tqdm(total=total_iter, desc="Progress", unit="iter")
df = pd.DataFrame()

totalRows = 0 
for pkl in pkl_files:
    

    with open(pkl, 'rb') as file:
        events = pickle.load(file)
        for i in range( len( events ) ):
            totalRows+=len(events[i])

            if 5.0009765625 != len(events[i])/1024:
                print( len(events[i])/1024)
                print( pkl , i)

# with open('raw_sub-20_class_1.pkl', 'rb') as file:
#     events = pickle.load(file)

#for i in range( len( events ) ):
#print(len(events[-1])/1024 )
totalRows

In [38]:
(49156479/(5.0009765625*1024))

9599.0

In [39]:
total_iter= len(pkl_files)
progress_bar = tqdm(total=total_iter, desc="Progress", unit="iter")
df = pd.DataFrame()
for pkl in pkl_files:
    

    with open(pkl, 'rb') as file:
        events = pickle.load(file)
    df_temp = pd.DataFrame()
    for event in events:
        data = pd.DataFrame(event.get_data())
        data = data.transpose()
        data['class'] = int(pkl[-5])
        data['subject'] = int(pkl[8:10])
        df_temp = pd.concat([df_temp, data], ignore_index=True)

    df = pd.concat([df, df_temp], ignore_index=True)
    progress_bar.update(1)

# Cerrar la barra de progreso al finalizar
progress_bar.close()

df































































rogress: 100%|███████████████████████████████████████████████████████████████████████| 57/57 [14:32<00:00, 15.31s/iter]

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,class,subject
0,-1.694066e-21,1.164670e-21,5.082198e-21,2.117582e-21,5.929231e-21,-4.235165e-22,2.117582e-21,7.411538e-21,5.929231e-21,3.388132e-21,...,2.752857e-21,4.235165e-22,1.270549e-21,8.470329e-22,-8.470329e-22,-8.470329e-22,1.270549e-21,-3.388132e-21,1,1
1,-2.667676e-07,4.663738e-07,-9.873639e-08,4.070683e-07,7.193366e-07,-1.726920e-07,1.164984e-06,1.833160e-06,-1.375810e-07,6.006594e-07,...,6.699455e-07,8.307907e-07,2.057207e-06,1.366691e-06,9.598713e-07,2.824962e-07,8.708696e-07,9.834886e-08,1,1
2,-5.117413e-07,9.166347e-07,-1.513974e-07,8.374756e-07,1.432947e-06,-3.440157e-07,2.296349e-06,3.636361e-06,-2.011210e-07,1.210862e-06,...,1.336863e-06,1.646250e-06,4.046963e-06,2.699336e-06,1.871922e-06,5.591864e-07,1.747167e-06,1.895870e-07,1,1
3,-7.146754e-07,1.336196e-06,-1.179943e-07,1.311546e-06,2.134137e-06,-5.121444e-07,3.362059e-06,5.380100e-06,-1.230970e-07,1.838368e-06,...,1.996809e-06,2.431605e-06,5.906240e-06,3.965018e-06,2.692056e-06,8.242168e-07,2.631829e-06,2.669365e-07,1,1
4,-8.594422e-07,1.713059e-06,3.017395e-08,1.844112e-06,2.814930e-06,-6.738802e-07,4.333417e-06,7.036312e-06,1.521477e-07,2.488366e-06,...,2.644093e-06,3.172290e-06,7.580012e-06,5.133712e-06,3.382960e-06,1.071344e-06,3.523968e-06,3.242323e-07,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49156474,-9.409061e-06,-3.141499e-06,-8.379883e-06,-5.933396e-06,-1.459741e-06,-2.178826e-06,-3.939842e-06,-3.359693e-06,-1.645382e-06,-3.918532e-06,...,-2.533642e-07,-6.198654e-06,-4.374413e-06,-3.310236e-06,-4.477206e-07,9.129486e-07,1.500918e-06,4.291450e-06,3,21
49156475,-7.393714e-06,-2.586678e-06,-6.632058e-06,-4.628604e-06,-1.136006e-06,-1.726994e-06,-3.211942e-06,-2.710958e-06,-1.310361e-06,-3.075795e-06,...,-2.779414e-07,-4.826026e-06,-3.390944e-06,-2.459670e-06,-2.911065e-07,7.111857e-07,1.058107e-06,3.180547e-06,3,21
49156476,-5.098529e-06,-1.840644e-06,-4.594245e-06,-3.174853e-06,-7.788930e-07,-1.198123e-06,-2.269999e-06,-1.902006e-06,-9.110509e-07,-2.118172e-06,...,-2.294897e-07,-3.305557e-06,-2.316180e-06,-1.627789e-06,-1.728027e-07,4.863884e-07,6.706145e-07,2.100165e-06,3,21
49156477,-2.601830e-06,-9.564161e-07,-2.350518e-06,-1.614942e-06,-3.962490e-07,-6.136069e-07,-1.174845e-06,-9.801187e-07,-4.669436e-07,-1.079815e-06,...,-1.284013e-07,-1.680103e-06,-1.175384e-06,-8.100845e-07,-8.007825e-08,2.468069e-07,3.243128e-07,1.043574e-06,3,21


In [40]:
#df.to_csv('/home/castrogaray-j/U_Winnipeg_OneDrive/Pattern recognition/BICT/raw/raw.csv', index = False)
with open('/home/castrogaray-j/U_Winnipeg_OneDrive/Pattern recognition/BICT/raw/raw_subjects_2.pkl', 'wb') as file:
    pickle.dump(df, file)